In [1]:
import os 
from dotenv import load_dotenv
load_dotenv() # loading all the environment variable

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Deepak\AgenticAIWorkshop\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore  = FAISS.from_documents(
    documents=doc_splits,
    embedding=embeddings
)

retreiver = vectorstore.as_retriever()

In [9]:
### Router

from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

from pydantic import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relavant datasource."""

    datasource: Literal["vectorstore", "web_search"] = Field(
        description="Given a user question choose to route it to web search or a vectorstore.",
    )

# LLM with function call
llm = ChatGroq(model='llama3-70b-8192')
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt
system="""You are an expert at routing a user question to a vectorstore or web search.
The vectorstore contains document related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for the questions on these topics, Otherwise, use web search."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt |structured_llm_router

print(question_router.invoke({"question": "What are the types of  agent memory"}))

datasource='vectorstore'


In [10]:
## Retrieval Grader

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

llm = ChatGroq(model='llama3-70b-8192')
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# prompt
system = """You are a grader assessing relevance of a retreived document to a user question. \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
It does not need to be a stringent test. The goal is to filter out erroneous retreivals. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

question = "agent memory" 
docs = retreiver.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [11]:
### Generate

from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGroq(model='llama3-70b-8192')

# Post-processing
# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

c:\Deepak\AgenticAIWorkshop\venv\Lib\site-packages\langsmith\client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


The agent memory consists of two components: short-term memory, which utilizes in-context learning to learn from the model, and long-term memory, which provides the capability to retain and recall infinite information over extended periods, often by leveraging an external vector store and fast retrieval.


In [12]:
### Hallucination Grader

# Data Model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_Score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

    # LLM with function call

llm = ChatGroq(model='llama3-70b-8192')
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """you are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts"""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation":generation})


GradeHallucinations(binary_Score='yes')

In [13]:
### Answer Grader

class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatGroq(model='llama3-70b-8192')
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
Give a binary score 'yes' or 'no' . 'Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),

    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

GradeAnswer(binary_score='yes')

In [15]:
### Question Re-writer

# LLM
llm = ChatGroq(model='llama3-70b-8192')

# Prompt
system = """You a question re-writer that converts an input question to a better version that is optimized \n
for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system),
        ("human", "Here si the initial quesiton: \n\n {question} \n Formulate an improved question."),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'Based on the initial question "agent memory", I\'m going to take a stab at reformulating it to make it more specific and informative, which should help with retrieval.\n\nHere\'s a rewritten question:\n\n"What are the different types of memory used in artificial intelligence agents?"\n\nThis revised question is more specific, as it:\n\n1. Adds context: "artificial intelligence agents")\n2. Asks for specific information: "types of memory"\n\nThis should help the vector store retrieval to focus on more relevant and accurate results.\n\nLet me know if you\'d like me to revise anything!'

In [ ]:
### search 

from dotenv import load_dotenv
load_dotenv()

os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [20]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    """
    Represents the state of our graph.
    
    Attributes:
    question: question
    generation: LLM generation
    documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [ ]:
from langchain.schema import Document

def retrieve(state):
    """
    Retrieve documents
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state(dict): New key added to state, documents, that contains retrieved documents
    """

    print("---Retrieve---")
    question = state["question"]

    # Retrieval
    documents = retreiver.invoke(question)
    return {"documents" : documents, "question": question}


def generate(state):
    """
    Generate answer
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """

    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def grade_documents(state):
    """
    Determines wheter the retrieved documents are relevant to the quesiton.
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == 'yes':
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)

        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def transform_query(state):
    """
    Transform the query to produce a better question.
    
    Args:
        state (dict): The current grapg state
        
    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    #Re- write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


def web_search(state):
    """
    Web search based on the re-phrased question.
    
    Args:
        state (dict): The current graph state
        
    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    quesiton = state["question"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)

    return {"documents": web_results, "question": question}


### Edges ##

def route_question(state):
    """
    Route question to web search or RAG.
    
    Args:
        state (dict): The current graph state
        
    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    if source.datasource == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "web_search"
    elif source.datasource == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"




